IMPORTS

In [1]:
from arcgis.gis import GIS
import tensorflow as tf
import os
from pathlib import Path



2026-01-14 16:02:19.473707: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-14 16:02:20.128400: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Verificar GPU
tf.config.list_physical_devices("GPU")


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

CONEXIÓN A AGOL Y DESCARGAR CONTENIDO

In [3]:
gis = GIS("home")

training_data = gis.content.get("81932a51f77b4d2d964218a7c5a4af17")

data_path = training_data.download(save_path="data")
data_path


'data/train_a_tensorflow-lite_model_for_identifying_plant_species.zip'

In [4]:
# Descomprimir
import zipfile

if data_path.endswith(".zip"):
    with zipfile.ZipFile(data_path, 'r') as zip_ref:
        zip_ref.extractall("data/extracted")


DETECTAR RUTAS DINÁMICAMENTE

In [5]:
ROOT = Path(os.getcwd())
extracted_dir = list(ROOT.rglob("extracted"))[0]

BASE_DIR = extracted_dir / "train" / "images"
BASE_DIR


PosixPath('/arcgis/data/extracted/train/images')

OBTENER CLASES E IMÁGENES IGNORANDO LOS XML

In [6]:
class_names = sorted([d.name for d in BASE_DIR.iterdir() if d.is_dir()])
num_classes = len(class_names)

image_paths = []
labels = []

for idx, class_name in enumerate(class_names):
    class_dir = BASE_DIR / class_name
    for img_path in class_dir.glob("*.jpg"):
        image_paths.append(str(img_path))
        labels.append(idx)

len(image_paths), num_classes


(39354, 100)

CREAR DATASET TF.DATA

In [7]:
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = tf.cast(img, tf.float32)
    return img, label

dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
dataset = dataset.shuffle(len(image_paths), seed=42)
dataset = dataset.map(load_image, num_parallel_calls=AUTOTUNE)

# Split 80 / 20
train_size = int(0.8 * len(image_paths))

train_ds = dataset.take(train_size)
val_ds   = dataset.skip(train_size)

train_ds = train_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_ds   = val_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)


2026-01-14 16:05:42.902162: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13775 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0001:00:00.0, compute capability: 7.5


AÑADIR DATOS

In [8]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
])


MODELO EfficientNetB0

In [10]:
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(*IMAGE_SIZE, 3),
    include_top=False,
    weights="imagenet"
)


16705208/16705208 [==============================] - 0s 0us/step


In [11]:
inputs = tf.keras.Input(shape=(256,256,3))

x = data_augmentation(inputs)
x = tf.keras.applications.efficientnet.preprocess_input(x)

x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(256, activation="relu")(x)
outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)


ENTRENAMIENTO FASE 1: TRANSFER LEARNING

In [12]:
base_model.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15
)


Epoch 1/15


2026-01-14 16:07:53.935795: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8700
2026-01-14 16:07:54.045391: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2026-01-14 16:07:54.046199: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2026-01-14 16:07:54.046230: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:109] Couldn't get ptxas version : FAILED_PRECONDITION: Couldn't get ptxas/nvlink version string: INTERNAL: Couldn't invoke ptxas --version
2026-01-14 16:07:54.046922: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2026-01-14 16:07:54.047076: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:318] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This mes

984/984 [==============================] - 116s 111ms/step - loss: 2.4639 - accuracy: 0.4057 - val_loss: 1.6292 - val_accuracy: 0.5703
Epoch 2/15
984/984 [==============================] - 108s 109ms/step - loss: 1.7553 - accuracy: 0.5435 - val_loss: 1.4101 - val_accuracy: 0.6270
Epoch 3/15
984/984 [==============================] - 108s 110ms/step - loss: 1.5437 - accuracy: 0.5873 - val_loss: 1.3141 - val_accuracy: 0.6495
Epoch 4/15
984/984 [==============================] - 108s 110ms/step - loss: 1.4064 - accuracy: 0.6207 - val_loss: 1.2126 - val_accuracy: 0.6647
Epoch 5/15
984/984 [==============================] - 108s 110ms/step - loss: 1.3152 - accuracy: 0.6413 - val_loss: 1.0936 - val_accuracy: 0.7005
Epoch 6/15
984/984 [==============================] - 108s 110ms/step - loss: 1.2391 - accuracy: 0.6597 - val_loss: 1.1005 - val_accuracy: 0.7005
Epoch 7/15
984/984 [==============================] - 108s 110ms/step - loss: 1.1861 - accuracy: 0.6718 - val_loss: 1.0281 - val_accura

FINE TUNING A

In [13]:
# Descongelar SOLO las últimas 40 capas
for layer in base_model.layers[:-40]:
    layer.trainable = False

for layer in base_model.layers[-40:]:
    layer.trainable = True


In [14]:
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)


history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15
)

Epoch 1/15
984/984 [==============================] - 114s 111ms/step - loss: 0.7156 - accuracy: 0.7884 - val_loss: 0.7606 - val_accuracy: 0.7967
Epoch 2/15
984/984 [==============================] - 108s 110ms/step - loss: 0.6540 - accuracy: 0.8093 - val_loss: 0.7220 - val_accuracy: 0.8045
Epoch 3/15
984/984 [==============================] - 108s 110ms/step - loss: 0.6340 - accuracy: 0.8136 - val_loss: 0.7169 - val_accuracy: 0.8023
Epoch 4/15
984/984 [==============================] - 108s 110ms/step - loss: 0.6073 - accuracy: 0.8223 - val_loss: 0.7110 - val_accuracy: 0.8000
Epoch 5/15
984/984 [==============================] - 108s 110ms/step - loss: 0.5979 - accuracy: 0.8219 - val_loss: 0.6976 - val_accuracy: 0.8107
Epoch 6/15
984/984 [==============================] - 108s 110ms/step - loss: 0.5722 - accuracy: 0.8294 - val_loss: 0.6749 - val_accuracy: 0.8169
Epoch 7/15
984/984 [==============================] - 108s 110ms/step - loss: 0.5723 - accuracy: 0.8309 - val_loss: 0.6799 -

FINE TUNING B

In [15]:
for layer in base_model.layers[:-80]:
    layer.trainable = False
for layer in base_model.layers[-80:]:
    layer.trainable = True


In [16]:
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

history_finetune_2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=8
)


Epoch 1/8
984/984 [==============================] - 114s 111ms/step - loss: 0.5015 - accuracy: 0.8508 - val_loss: 0.6511 - val_accuracy: 0.8196
Epoch 2/8
984/984 [==============================] - 108s 110ms/step - loss: 0.4965 - accuracy: 0.8524 - val_loss: 0.6345 - val_accuracy: 0.8243
Epoch 3/8
984/984 [==============================] - 108s 110ms/step - loss: 0.4948 - accuracy: 0.8536 - val_loss: 0.6325 - val_accuracy: 0.8258
Epoch 4/8
984/984 [==============================] - 108s 110ms/step - loss: 0.4949 - accuracy: 0.8508 - val_loss: 0.6302 - val_accuracy: 0.8223
Epoch 5/8
984/984 [==============================] - 112s 114ms/step - loss: 0.4827 - accuracy: 0.8553 - val_loss: 0.6670 - val_accuracy: 0.8132
Epoch 6/8
984/984 [==============================] - 108s 110ms/step - loss: 0.4845 - accuracy: 0.8525 - val_loss: 0.6199 - val_accuracy: 0.8314
Epoch 7/8
984/984 [==============================] - 108s 110ms/step - loss: 0.4839 - accuracy: 0.8569 - val_loss: 0.6286 - val_ac

GUARDAR EL MODELO

In [17]:
model.save("plant_species_tf")


INFO:tensorflow:Assets written to: plant_species_tf/assets


INFO:tensorflow:Assets written to: plant_species_tf/assets


CONVERSION A TENSORFLOW LITE

In [18]:
converter = tf.lite.TFLiteConverter.from_saved_model("plant_species_tf")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()

with open("plant_species.tflite", "wb") as f:
    f.write(tflite_model)


2026-01-14 17:17:08.330143: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2026-01-14 17:17:08.330193: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2026-01-14 17:17:08.335060: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: plant_species_tf
2026-01-14 17:17:08.369847: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2026-01-14 17:17:08.369890: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: plant_species_tf
2026-01-14 17:17:08.424299: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2026-01-14 17:17:08.450250: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2026-01-14 17:17:08.968586: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: plant_species_tf
2026-01